To dos  
1) Handle custom bin widths of histograms for categorical datasets  

In [1]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col

# ee.Authenticate()
ee.Initialize(project = "dri-apps")

Map = geemap.Map()

## Define Parameters and Load datasets

In [12]:
# ------------------------------------- Define parameters -----------------------------------------------

# Define time period to export
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2010, 1, 1)


# -------------------------------- Define input Image Collection ----------------------------------------

# Define input dataset
# See dictionary below for list of input datasets
in_ic_name = 'GridMET'

# Define variable from dataset
# See dictionary below for variables available for each dataset
var_name = 'precip'


# ------------------------------- Define input Feature Collection ---------------------------------------

# Define input path for Feature Collection
in_fc_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'
in_fc = ee.FeatureCollection(in_fc_path)

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-108.4020, 38.7855], [-108.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 38.7855]]], None, False);
# in_fc = in_fc.filterBounds(geometry)


# ------------------------------ Define mask, if applicable --------------------------------------------

# Apply mask for ownership, landcover, or other variables. Must be binary mask.
mask = True
mask_path = 'projects/dri-apps/assets/blm-admin/blm-natl-admu-sma-binary'


# ---------------------------- Additional parameters derived from above --------------------------------

# Define input Image Collection variables using dataset dictionary
in_ic_dict = {'GridMET_Drought': {'in_ic_paths': ['GRIDMET/DROUGHT'],
                                  'var_names': ['Long_Term_Drought_Blend', 'Short_Term_Drought_Blend'],
                                  'var_type': 'Categorical'},
              'GridMET': {'in_ic_paths': ['IDAHO_EPSCOR/GRIDMET'],
                          'var_names': ['precip', 'tmmn', 'tmmx', 'eto', 'vpd', 'windspeed', 'srad'],
                          'var_type': 'Continuous'},
              'RAP_Cover': {'in_ic_paths': ['projects/rap-data-365417/assets/vegetation-cover-v3'],
                            'var_names': ['AFG', 'BGR', 'LTR', 'PFG', 'SHR', 'TRE'],
                            'var_type': 'Continuous'},
              'RAP_Production': {'in_ic_paths': ['projects/rap-data-365417/assets/npp-partitioned-v3'],
                                 'var_names': ['afgAGB', 'pfgAGB', 'herbaceousAGB'],
                                 'var_type': 'Continuous'},
              'RAP16_Production': {'in_ic_paths': ['projects/rap-data-365417/assets/npp-partitioned-16day-v3'],
                                   'var_names': ['afgAGB', 'pfgAGB', 'herbaceousAGB'],
                                   'var_type': 'Continuous'},
              'USDM': {'in_ic_paths': ['projects/climate-engine/usdm/weekly'],
                       'var_names': ['drought'],
                       'var_type': 'Categorical'},
              'MOD11_LST': {'in_ic_paths': ['MODIS/061/MOD11A2'],
                            'var_names': ['LST_Day_1km'],
                            'var_type': 'Continuous'},
              'LS_NDVI': {'in_ic_paths': ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2'],
                          'var_names': ['NDVI'],
                          'var_type': 'Continuous'},
              'MOD16_ET': {'in_ic_paths': ['MODIS/006/MOD16A2'],
                           'var_names': ['ET', 'PET'],
                           'var_type': 'Continuous'},
              'MTBS': {'in_ic_paths': ['projects/climate-engine-pro/assets/mtbs_mosaics_annual'],
                       'var_names': ['Severity'],
                       'var_type': 'Categorical'}}

# Define properties for variables in dictionary
var_dict = {'Long_Term_Drought_Blend': {'units': 'drought'},
            'Short_Term_Drought_Blend': {'units': 'drought'},
            'precip': {'units': 'mm'},
            'tmmn': {'units': 'degrees C'},
            'tmmx': {'units': 'degrees C'},
            'eto': {'units': 'mm'},
            'vpd': {'units': 'kPa'},
            'windspeed': {'units': 'm/s'},
            'srad': {'units': 'W/m^2'},
            'AFG': {'units': '% cover'},
            'BGR': {'units': '% cover'},
            'LTR': {'units': '% cover'},
            'PFG': {'units': '% cover'},
            'SHR': {'units': '% cover'},
            'TRE': {'units': '% cover'},
            'afgAGB': {'units': 'lbs/acre'},
            'pfgAGB': {'units': 'lbs/acre'},
            'herbaceousAGB': {'units': 'lbs/acre'},
            'drought': {'units': 'drought'},
            'LST_Day_1km': {'units': 'degrees C'},
            'NDVI': {'units': 'unitless'},
            'ET': {'units': 'mm'},
            'PET': {'units': 'mm'},
            'Severity': {'units': 'fire severity'}}

# Define land unit names
if(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-grazing-allotment-polygons'):
    land_unit_long = 'BLM_Natl_Grazing_Allotment_Polygons'
    land_unit_short = 'BLM_Allotments'
    in_fc_id = 'ALLOT_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-stateoffice-polygons'):
    land_unit_long = 'BLM_Natl_FieldOffice_Polygons'
    land_unit_short = 'BLM_FieldOffices'
    in_fc_id = 'FO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-districtoffice-polygons'):
    land_unit_long = 'BLM_Natl_DistrictOffice_Polygons'
    land_unit_short = 'BLM_DistrictOffices'
    in_fc_id = 'DO_ID'
elif(in_fc_path == 'projects/dri-apps/assets/blm-admin/blm-natl-admu-fieldoffice-polygons'):
    land_unit_long = 'BLM_Natl_StateOffice_Polygons'
    land_unit_short = 'BLM_StateOffices'
    in_fc_id = 'SO_ID'
    
# Pull out additional variables needed to run exports
in_ic_paths = in_ic_dict.get(in_ic_name).get('in_ic_paths')
in_ic_res = ee.Number(ee.ImageCollection(in_ic_paths[0]).first().projection().nominalScale()).round().getInfo()
var_type = in_ic_dict.get(in_ic_name).get('var_type')
var_units = var_dict.get(var_name).get('units')
out_path = f"projects/dri-apps/assets/blm-database/{land_unit_short.replace('_', '').lower()}-{in_ic_name.replace('_', '').lower()}-{var_name.replace('_', '').lower()}"

## Create database image collection and append time-series images

In [6]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'mask': mask}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    # Apply ID image function to input feature collection
    out_list = eedb_cor.generate_id_img(in_fc = in_fc, in_fc_id = in_fc_id)
    out_i = ee.Image(out_list.get(0))
    out_fc = ee.FeatureCollection(out_list.get(1))
    
    # Generate empty Image Collection asset to append images
    os.system(f"earthengine create collection {out_path}")
    
    # Export ID image to new Image Collection
    task = ee.batch.Export.image.toAsset(
        image = out_i.set(properties),
        description = f"initialize - {land_unit_short.replace('_', '').lower()} {in_ic_name.replace('_', '').lower()} {var_name.replace('_', '').lower()} - id",
        assetId = out_path + '/0_id',
        region = out_fc.geometry().buffer(20),
        scale = 22.264,
        maxPixels = 1e13)
    task.start()

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:
    
    print(f"Appending to Image Collection for dates {start_date} - {end_date}")
    
    # ----- Preprocess input Image Collection based on path -----
    
    if in_ic_paths == ['GRIDMET/DROUGHT']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_gm_drought(in_ic_paths, var_name, start_date, end_date)
        
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()
        
    elif in_ic_paths == ['IDAHO_EPSCOR/GRIDMET']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_gm(in_ic_paths, var_name, start_date, end_date)
    
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()
    
    elif in_ic_paths == ['projects/rap-data-365417/assets/vegetation-cover-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-16day-v3']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_rap(in_ic_paths, var_name, start_date, end_date)
    
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()

    elif in_ic_paths == ['projects/climate-engine/usdm/weekly']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_usdm(in_ic_paths, var_name, start_date, end_date)
    
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()

    elif in_ic_paths == ['MODIS/061/MOD11A2']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_modlst(in_ic_paths, var_name, start_date, end_date)
        
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()
    
    elif in_ic_paths == ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2']:
    
        # Run function to pre-process the Landsat SR NDVI data
        in_i = eedb_col.preprocess_lsndvi(in_ic_paths, var_name, start_date, end_date, in_fc)
    
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()
        
    elif in_ic_paths == ['MODIS/006/MOD16A2']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_modet(in_ic_paths, var_name, start_date, end_date)
        
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()
    
    elif in_ic_paths == ['projects/climate-engine-pro/assets/mtbs_mosaics_annual']:
    
        # Run function to pre-process the GridMET drought data
        in_i = eedb_col.preprocess_mtbs(in_ic_paths, var_name, start_date, end_date)
    
        # Get list of date strings from image
        in_dates = in_i.bandNames().getInfo()


    # ---------------------------- Iterate over in_dates with functions ---------------------------------
        
    for in_date in in_dates:
    
        print('Running ' + in_date)
        
        # Conditionally apply mask to images
        if mask == True:
            
            # Select date band for single date
            in_i_date = in_i.select([in_date]).updateMask(ee.Image(mask_path))
            
        elif mask == False:
            
            # Select date band for single date
            in_i_date = in_i.select([in_date])
    
        # Create dictionary of properties        
        properties = {'system:index': in_date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                      'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                      'var_name': var_name, 'var_units': var_units, 'mask': mask}
        if mask == True:
            properties['mask_path'] = mask_path
        elif mask == False:
            properties['mask_path'] = 'None'

        if var_type == 'Continuous':
        
            # Run function to get time-series statistics for input feature collection
            out_fc = eedb_cor.img_to_pts_continuous(in_i_date, in_fc)

            # Convert centroid time-series to image collection time-series
            out_i = eedb_cor.pts_to_img_continuous(in_fc = out_fc)
        
        elif var_type == 'Categorical':
        
            # Run function to get time-series statistics for input feature collection for continuous variables
            out_fc = eedb_cor.img_to_pts_categorical(in_i_date, in_fc, in_ic_name = in_ic_name)
       
            # Convert centroid time-series to image collection time-series
            out_i = eedb_cor.pts_to_img_categorical(in_fc = out_fc, in_ic_name = in_ic_name)
        
        # Create out region for export
        out_region = out_fc.geometry().buffer(20)
        
        # Export the image
        eedb_cor.export_img(out_i = out_i, out_region = out_region, out_path = out_path, out_id = in_date, properties = properties)

{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-06T16:03:24.087333Z"
}
{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-06T16:03:24.087333Z"
}
Appending to Image Collection for dates 1990-01-01 00:00:00 - 2000-01-01 00:00:00
Running 19900105
Running 19900110
Running 19900115
Running 19900120
Running 19900125
Running 19900130
Running 19900204
Running 19900209
Running 19900214
Running 19900219
Running 19900224
Running 19900301
Running 19900306
Running 19900311
Running 19900316
Running 19900321
Running 19900326
Running 19900331
Running 19900405
Running 19900410
Running 19900415
Running 19900420
Running 19900425
Running 19900430
Running 199005

Running 19960224
Running 19960229
Running 19960305
Running 19960310
Running 19960315
Running 19960320
Running 19960325
Running 19960330
Running 19960404
Running 19960409
Running 19960414
Running 19960419
Running 19960424
Running 19960429
Running 19960504
Running 19960509
Running 19960514
Running 19960519
Running 19960524
Running 19960529
Running 19960603
Running 19960608
Running 19960613
Running 19960618
Running 19960623
Running 19960628
Running 19960703
Running 19960708
Running 19960713
Running 19960718
Running 19960723
Running 19960728
Running 19960802
Running 19960807
Running 19960812
Running 19960817
Running 19960822
Running 19960827
Running 19960901
Running 19960906
Running 19960911
Running 19960916
Running 19960921
Running 19960926
Running 19961001
Running 19961006
Running 19961011
Running 19961016
Running 19961021
Running 19961026
Running 19961031
Running 19961105
Running 19961110
Running 19961115
Running 19961120
Running 19961125
Running 19961130
Running 19961205
Running 199612

### Attempt to loop over years to produce exports to reduce pre-processing burden

In [ ]:
# If there is no Image Collection asset at the out_path create one and export ID image
if os.system(f"earthengine asset info {out_path}") == 256:

    print("Initializing Image Collection by creating EE asset and exporting ID image")
    
    # Create dictionary of properties
    properties = {'system:index': '0_id', 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path, 
                  'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type, 
                  'var_name': var_name, 'var_units': var_units, 'mask': mask}
    if mask == True:
            properties['mask_path'] = mask_path
    elif mask == False:
            properties['mask_path'] = 'None'
    
    # Apply ID image function to input feature collection
    out_list = eedb_cor.generate_id_img(in_fc = in_fc, in_fc_id = in_fc_id)
    out_i = ee.Image(out_list.get(0))
    out_fc = ee.FeatureCollection(out_list.get(1))
    
    # Generate empty Image Collection asset to append images
    os.system(f"earthengine create collection {out_path}")
    
    # Export ID image to new Image Collection
    task = ee.batch.Export.image.toAsset(
        image = out_i.set(properties),
        description = f"initialize - {land_unit_short.replace('_', '').lower()} {in_ic_name.replace('_', '').lower()} {var_name.replace('_', '').lower()} - id",
        assetId = out_path + '/0_id',
        region = out_fc.geometry().buffer(20),
        scale = 22.264,
        maxPixels = 1e13)
    task.start()

    
# If there is an Image Collection asset at the out_path export time-series images
elif os.system(f"earthengine asset info {out_path}") == 0:
    
    for year in range(start_date.year, end_date.year):
        
        start_date_inner = datetime.datetime(year, 1, 1)
        end_date_inner = datetime.datetime(year+1, 1, 1)
        
        print(f"Appending to Image Collection for dates {start_date_inner} - {end_date_inner}")
        
        # ----- Preprocess input Image Collection based on path -----
        
        if in_ic_paths == ['GRIDMET/DROUGHT']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_gm_drought(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
            
        elif in_ic_paths == ['IDAHO_EPSCOR/GRIDMET']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_gm(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['projects/rap-data-365417/assets/vegetation-cover-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-v3'] or in_ic_paths == ['projects/rap-data-365417/assets/npp-partitioned-16day-v3']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_rap(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
        elif in_ic_paths == ['projects/climate-engine/usdm/weekly']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_usdm(in_ic_paths, var_name, start_date_inner, end_date_inner)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
        elif in_ic_paths == ['MODIS/061/MOD11A2']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_modlst(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['LANDSAT/LT05/C02/T1_L2', 'LANDSAT/LE07/C02/T1_L2', 'LANDSAT/LC08/C02/T1_L2', 'LANDSAT/LC09/C02/T1_L2']:
        
            # Run function to pre-process the Landsat SR NDVI data
            in_i = eedb_col.preprocess_lsndvi(in_ic_paths, var_name, start_date_inner, end_date_inner, in_fc)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
            
        elif in_ic_paths == ['MODIS/006/MOD16A2']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_modet(in_ic_paths, var_name, start_date_inner, end_date_inner)
            
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
        
        elif in_ic_paths == ['projects/climate-engine-pro/assets/mtbs_mosaics_annual']:
        
            # Run function to pre-process the GridMET drought data
            in_i = eedb_col.preprocess_mtbs(in_ic_paths, var_name, start_date, end_date)
        
            # Get list of date strings from image
            in_dates = in_i.bandNames().getInfo()
    
    
        # ---------------------------- Iterate over in_dates with functions ---------------------------------
            
        for in_date in in_dates:
        
            print('Running ' + in_date)
            
            # Conditionally apply mask to images
            if mask == True:
                
                # Select date band for single date
                in_i_date = in_i.select([in_date]).updateMask(ee.Image(mask_path))
                
            elif mask == False:
                
                # Select date band for single date
                in_i_date = in_i.select([in_date])
        
            # Create dictionary of properties        
            properties = {'system:index': in_date, 'land_unit_long': land_unit_long, 'land_unit_short': land_unit_short, 'in_fc_path': in_fc_path,\
                          'in_fc_id': in_fc_id, 'in_ic_paths': in_ic_paths[0], 'in_ic_name': in_ic_name, 'in_ic_res': in_ic_res, 'var_type': var_type,\
                          'var_name': var_name, 'var_units': var_units, 'mask': mask}
            if mask == True:
                properties['mask_path'] = mask_path
            elif mask == False:
                properties['mask_path'] = 'None'
    
            if var_type == 'Continuous':
            
                # Run function to get time-series statistics for input feature collection
                out_fc = eedb_cor.img_to_pts_continuous(in_i_date, in_fc)
    
                # Convert centroid time-series to image collection time-series
                out_i = eedb_cor.pts_to_img_continuous(in_fc = out_fc)
            
            elif var_type == 'Categorical':
            
                # Run function to get time-series statistics for input feature collection for continuous variables
                out_fc = eedb_cor.img_to_pts_categorical(in_i_date, in_fc, in_ic_name = in_ic_name)
           
                # Convert centroid time-series to image collection time-series
                out_i = eedb_cor.pts_to_img_categorical(in_fc = out_fc, in_ic_name = in_ic_name)
            
            # Create out region for export
            out_region = out_fc.geometry().buffer(20)
            
            # Export the image
            eedb_cor.export_img(out_i = out_i, out_region = out_region, out_path = out_path, out_id = in_date, properties = properties)

{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-06T16:28:28.631090Z"
}
{
  "id": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "name": "projects/dri-apps/assets/blm-database/blmdistrictoffices-gridmet-precip",
  "type": "IMAGE_COLLECTION",
  "updateTime": "2023-06-06T16:28:28.631090Z"
}
Appending to Image Collection for dates 2000-01-01 00:00:00 - 2001-01-01 00:00:00
Running 20000105
Running 20000110
Running 20000115
Running 20000120
Running 20000125
Running 20000130
Running 20000204
Running 20000209
Running 20000214
Running 20000219
Running 20000224
Running 20000229
Running 20000305
Running 20000310
Running 20000315
Running 20000320
Running 20000325
Running 20000330
Running 20000404
Running 20000409
Running 20000414
Running 20000419
Running 20000424
Running 20000429
Running 200005